In [ ]:
# Importer les bibliothèques
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Charger les données
data = pd.read_csv("../data/classifieds_data.csv")


data.head()

In [1]:
# Info des datas
data.info()

NameError: name 'data' is not defined

In [2]:
# Prétraitement des données
# Gestion des valeurs manquantes pour les colonnes numériques
# Supposons que 'pricePerSquareMeter' et 'value' sont les principales caractéristiques d'intérêt
data.fillna({'pricePerSquareMeter': data['pricePerSquareMeter'].median(), 'value': data['value'].median()}, inplace=True)

NameError: name 'data' is not defined

In [3]:
# Analyse exploratoire
# Statistiques descriptives pour les caractéristiques numériques
print(data[['pricePerSquareMeter', 'value', 'fees']].describe())

NameError: name 'data' is not defined

In [4]:
# Sélection des caractéristiques pour le clustering
features = data[['pricePerSquareMeter', 'value']]
scaler = StandardScaler()
kmeans = KMeans(n_clusters=3, random_state=42)  # Ajustez le nombre de clusters selon vos données

# Pipeline de prétraitement et de clustering
pipeline = make_pipeline(scaler, kmeans)
pipeline.fit(features)

# Prédire les clusters pour déterminer les groupes d'annonces
clusters = pipeline.predict(features)

# Ajouter les clusters aux données
data['cluster'] = clusters

# Analyse des clusters pour construire la requête Elasticsearch
# Calcul des moyennes pour 'pricePerSquareMeter' et 'value' par cluster
cluster_summary = data.groupby('cluster')[['pricePerSquareMeter', 'value']].mean()

NameError: name 'data' is not defined

In [5]:
plt.scatter(data['pricePerSquareMeter'], data['value'], c=data['cluster'], cmap='viridis')
plt.xlabel('Prix au mètre carré standardisé')
plt.ylabel('Valeur standardisée')
plt.show()

NameError: name 'plt' is not defined

In [6]:
cluster_summary

NameError: name 'cluster_summary' is not defined

In [ ]:
# Créer une fonction pour générer des requêtes Elasticsearch basées sur les clusters
def generate_es_query(cluster_summary, cluster_id):
    cluster = cluster_summary.loc[cluster_id]
    query = {
      "query": {
        "bool": {
          "filter": [
            {"range": {"value": {"gte": cluster['value'] * 0.8, "lte": cluster['value'] * 1.2}}},
            {"range": {"pricePerSquareMeter": {"gte": cluster['pricePerSquareMeter'] * 0.8, "lte": cluster['pricePerSquareMeter'] * 1.2}}}
          ]
        }
      }
    }
    return query

# Exemple d'utilisation : Générer une requête pour le premier cluster
example_query = generate_es_query(cluster_summary, 0)
print(example_query)